# 卷积神经网络
网络结构：
* 输入层 n x 784
* 卷积层：3 x 3 x 1
* 池化层：2 x 2
* 卷积层：3 x 3 x 64
* 池化层：2 x 2
* 全连接层：1024
* 全连接层：10

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 加载数据
mnist = input_data.read_data_sets('mnist_data',one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels

Extracting mnist_data\train-images-idx3-ubyte.gz
Extracting mnist_data\train-labels-idx1-ubyte.gz
Extracting mnist_data\t10k-images-idx3-ubyte.gz
Extracting mnist_data\t10k-labels-idx1-ubyte.gz


In [3]:
# 设置参数
n_input = 784
n_output = 10
stddev = 0.1
# 网络权重初始化
weights = {
    'wc1' : tf.Variable(tf.random_normal([3,3,1,64],stddev=stddev)),
    'wc2' : tf.Variable(tf.random_normal([3,3,64,128],stddev=stddev)),
    'wdl' : tf.Variable(tf.random_normal([7*7*128,1024],stddev=stddev)),
    'wd2' : tf.Variable(tf.random_normal([1024,n_output],stddev=stddev))
}
biases = {
    'bc1' : tf.Variable(tf.random_normal([64],stddev=stddev)),
    'bc2' : tf.Variable(tf.random_normal([128],stddev=stddev)),
    'bd1' : tf.Variable(tf.random_normal([1024],stddev=stddev)),
    'bd2' : tf.Variable(tf.random_normal([n_output],stddev=stddev))
}

In [4]:
#卷积计算
def conv_basic(_input,_w,_b,_keepration):
    # 输入数据维度的转换,n,h,w,c
    _input_r = tf.reshape(_input,shape=[-1,28,28,1])
    # 1、卷积层
    # 卷积操作
    _conv1 = tf.nn.conv2d(_input_r,
                          _w['wc1'],
                          strides=[1,1,1,1],
                          padding='SAME')
    _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1,_b['bc1']))
    # 池化操作
    _pool1 = tf.nn.max_pool(_conv1,
                            ksize=[1,2,2,1],
                           strides=[1,2,2,1],
                           padding='SAME')
    _pool_dr1 = tf.nn.dropout(_pool1,_keepration)
    # 2、卷积层
    _conv2 = tf.nn.conv2d(_pool_dr1,
                         _w['wc2'],
                         strides=[1,1,1,1],
                         padding='SAME')
    _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2,_b['bc2']))
    # 池化操作
    _pool2 = tf.nn.max_pool(_conv2,
                            # ksize：池化窗口的大小，[1, height, width, 1]，因为我们不想在batch和channels上做池化，所以这两个维度设为了1
                            ksize=[1,2,2,1],
                            # strides：和卷积类似，窗口在每一个维度上滑动的步长，一般也是[1, stride,stride, 1]
                            strides=[1,2,2,1],
                            # padding：和卷积类似，可以取'VALID' 或者'SAME'，
                            # padding = 'SAME' 时，输出并不一定和原图size一致，但会保证覆盖原图所有像素，不会舍弃边上的莫些元素
                            # padding = 'VALID' 时，输出的size总比原图的size小，有时不会覆盖原图所有元素(既，可能舍弃边上的某些元素).
                            padding='SAME')
    _pool_dr2 = tf.nn.dropout(_pool2,_keepration)
    
    # 3、全连接层
    _dense1 = tf.reshape(_pool_dr2,[-1,6272])
#     _dense1 = tf.reshape(_pool_dr2, [-1, _w['wdl'].get_shape().as_list()[0]])
    print('w 的 wd1的维度为：',_w['wdl'].get_shape().as_list()[0])
    # 全连接层1
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1,
                                       _w['wdl']),
                             _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1,_keepration)
    # 全连接层2
    _out = tf.add(tf.matmul(_fc_dr1,_w['wd2']),_b['bd2'])
    
    # 返回值
    result = {'input_r':_input_r,
             'conv1':_conv1,
             'conv2':_conv2,
             'pool1':_pool1,
              'pool1_dr1':_pool_dr1,
             'pool2':_pool2,
             'pool2_dr2':_pool_dr2,
             'dense1':_dense1,
             'fc1':_fc1,
             'fc_dr1':_fc_dr1,
             'out':_out}
    return result 
    
    

In [5]:
# 构建卷积神经网络图
x = tf.placeholder(tf.float32,[None,n_input])
y = tf.placeholder(tf.float32,[None,n_output])
keep_ratio = tf.placeholder(tf.float32)

_pred = conv_basic(x,
                   weights,
                   biases,
                   keep_ratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=_pred))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1),tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(_corr,tf.float32))


w 的 wd1的维度为： 6272


In [6]:
training_epochs = 20
batch_size = 10
display_step = 1
kp_ratio = 1.0

config = tf.ConfigProto()  
# config.gpu_options.per_process_gpu_memory_fraction = 0.95  # 程序最多只能占用指定gpu 95%的显存  
config.gpu_options.allow_growth = True      #程序按需申请内存  
# 迭代
with tf.Session(config=config ) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = (mnist.train.num_examples // batch_size)
        # 批次循环
        for i in range(total_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size=batch_size)
            feeds = {x:batch_x,
                     y:batch_y,
                     keep_ratio:kp_ratio}
            # 进行一次梯度下降训练
            sess.run(optm,feed_dict=feeds)
             # 计算本次的代价，并累计
            avg_cost += sess.run(cost,feed_dict=feeds)
        avg_cost = avg_cost / total_batch
         # 显示中间结果
        if (epoch + 1) % display_step == 0:
            print('Epoch: %03d / %03d cost: %0.9f' 
                   % ( epoch,training_epochs,avg_cost ))
            # 计算训练集准确度，和测试集准确度
            feeds = {x:batch_x,y:batch_y,keep_ratio:kp_ratio} 
            train_acc = sess.run(accr,feed_dict=feeds)
            print('\\t训练集准确度：%.3f' ,train_acc)
            feeds = {x:mnist.test.images,y:mnist.test.labels,keep_ratio:kp_ratio} 
            test_acc = sess.run(accr,feed_dict=feeds)
            print('\\t测试集准确度：%.3f' ,test_acc)
    pass

print('Close')
    

Epoch: 000 / 020 cost: 0.085929349
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9859
Epoch: 001 / 020 cost: 0.018208779
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9857
Epoch: 002 / 020 cost: 0.009883917
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9845
Epoch: 003 / 020 cost: 0.005562508
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9899
Epoch: 004 / 020 cost: 0.004520011
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9909
Epoch: 005 / 020 cost: 0.003054108
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9868
Epoch: 006 / 020 cost: 0.002508840
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9894
Epoch: 007 / 020 cost: 0.001803466
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9903
Epoch: 008 / 020 cost: 0.001486689
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9901
Epoch: 009 / 020 cost: 0.001271263
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9875
Epoch: 010 / 020 cost: 0.000867418
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9913
Epoch: 011 / 020 cost: 0.001010538
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9908
Epoch: 012 / 020 cost: 0.000755232
\t训练集准确度：%.3f 1.0
\t测试集准确度：%.3f 0.9903
Epoch: 013 / 020 cost: 0.000867306
\t训